# Análise e otimização inicial dos modelos

Nesse notebook, ambos os modelos requisitados (randomForest e XGBoost) serão testados e ocorrerá uma otimização dos seus parâmetros.

Acerca dos pipelines usados:
- As classes de classificadores utilizadas serão o randomForestClassifier (do pacote sklearn) e o XGBClassifier (do pacote XGBoost).
- Ambos os pipelines contam com discretização dos valores por meio da classe KBinsDiscretizer, que conhecidamente auxilia a performance de modelos baseados em árvores de decisão. Essa discretização, quando usada com a estratégia "kmeans", agrupa valores próximos em uma mesma categoria.
- Ao longo da otimização serão testados valores diferentes para os parâmetros n_bins (3 a 10) e encode (ordinal ou one-hot encoding).

Acerca da otimização:
- A otimização será feita utilizando RandomizedSearchCV. Embora usar gridsearch (que representa uma busca exaustiva) ou outras ferramentas de otimização e gerenciamento (como optuna) poderiam trazer resultados ótimos, o tempo de entrega do projeto foi priorizado.
- Os melhores modelos serão determinados pelo valores de "accuracy" encontrados.
- Os melhores parâmetros (aqueles que geraram os melhores modelos) serão salvos e utilizados como valores default durante a construção da CLI para treinamento rápido dos modelos.


In [7]:
%pip install scikit-learn
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join('./dataprep'))
sys.path.insert(0, module_path)

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder
from sklearn.pipeline import Pipeline
from dataprep import process_dataset
from xgboost import XGBClassifier

In [9]:
dataset_file_path = "./dataset/coleta.txt"

label_legend = {
    "Downstairs": 0,
    "Jogging": 1,
    "Sitting": 2,
    "Standing": 3,
    "Upstairs": 4,
    "Walking": 5
}

X_train, y_train, X_test, y_test, X_valid, y_valid = process_dataset(dataset_file_path, label_legend)

/tmp/ipykernel_780179/2016628047.py:12: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  X_train, y_train, X_test, y_test, X_valid, y_valid = process_dataset(dataset_file_path, label_legend)


# Otimizando Random Forest
Parâmetros escolhidos para otimização:
1. n_estimators
2. criterion
3. max_depth
4. bootstrap

# Configurações de busca

In [10]:
iterations = 10 # How many parameter settings each search will sample
n_jobs = -1 # How many cores will it try to use (-1 will use all available cores)
folds = 5 # How many folds will exist in CV score
scoring = "accuracy"
random_seed = 42

In [11]:
rf_classifier = Pipeline([
    ('discretizer', KBinsDiscretizer(strategy="kmeans")),
    ('clf', RandomForestClassifier())
])

rf_parameters = {
    'discretizer__n_bins': list(range(3, 10)),
    'discretizer__encode': ["ordinal", "onehot"],
    'clf__n_estimators': list(range(10, 200)),
    'clf__criterion': ["gini", "entropy", "log_loss"],
    'clf__max_depth': [None] + list(range(1, 5)),
    'clf__bootstrap': [True, False]
}

rf_randomized_search = RandomizedSearchCV(
    rf_classifier,
    rf_parameters,
    scoring=scoring,
    random_state=random_seed,
    n_iter=iterations,
    n_jobs=n_jobs,
    cv=folds
)
rf_randomized_search.fit(X_train, y_train.values.ravel())

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('discretizer',
                                              KBinsDiscretizer(strategy='kmeans')),
                                             ('clf',
                                              RandomForestClassifier())]),
                   n_jobs=-1,
                   param_distributions={'clf__bootstrap': [True, False],
                                        'clf__criterion': ['gini', 'entropy',
                                                           'log_loss'],
                                        'clf__max_depth': [None, 1, 2, 3, 4],
                                        'clf__n_estimators': [10, 11, 12, 13,
                                                              14, 15, 16, 17,
                                                              18, 19, 20, 21,
                                                              22, 23, 24, 25,
                                                              26, 27, 28, 29,
                                                              30, 31, 32, 33,
                                                              34, 35, 36, 37,
                                                              38, 39, ...],
                                        'discretizer__encode': ['ordinal',
                                                                'onehot'],
                                        'discretizer__n_bins': [3, 4, 5, 6, 7,
                                                                8, 9]},
                   random_state=42, scoring='accuracy')

In [12]:
rf_best_params = rf_randomized_search.best_params_

print("Best parameters for random forest")
for param_name in rf_parameters.keys():
    print(f"{param_name}: {rf_best_params[param_name]}")

print("Stats for optimized random forest")

print("training dataset:")
rf_train_pred = rf_randomized_search.predict(X_train)
precision, recall, f1, _ = precision_recall_fscore_support(y_train, rf_train_pred)
print("Precision:", precision)
print("recall:", recall)
print("f1:", f1)
print("")

print("testing dataset")
rf_test_pred = rf_randomized_search.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, rf_test_pred)
print("Precision:", precision)
print("recall:", recall)
print("f1:", f1)
print("")

Best parameters for random forest
discretizer__n_bins: 9
discretizer__encode: ordinal
clf__n_estimators: 130
clf__criterion: entropy
clf__max_depth: None
clf__bootstrap: False
Stats for optimized random forest
training dataset:


/home/alps2/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: [0.34530036 0.6692229  0.79203175 0.42346644 0.         0.54318468]
recall: [0.01569655 0.65931609 0.90516894 0.58072522 0.         0.79459984]
f1: [0.03002808 0.66423256 0.84482942 0.48978228 0.         0.64526753]

testing dataset
Precision: [0.3369235  0.66668469 0.79202279 0.41891788 0.         0.54342545]
recall: [0.01534032 0.65802645 0.90507801 0.57439802 0.         0.79427756]
f1: [0.02934456 0.66232727 0.84478472 0.48448958 0.         0.64533105]



/home/alps2/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Otimizando Extreme Gradient Boost

Parâmetros escolhidos para otimização:
1. eta
2. min_child_weight
3. max_depth

O classificador sempre tem o parâmetro "objective" definido como "multi:softmax", de forma que ele retorne a classe predita (ao invés da probabilidade de cada classe). Por causa disso, é necessário também configurar o parâmetro "num_class" com o número de classes diferentes que existem (6).

Esse bloco de código inclui um label encoder que traduz ['Downstairs' 'Jogging' 'Sitting' 'Standing' 'Upstairs' 'Walking'] para [0 1 2 3 4 5]. Isso é necessário para a execução do XGBoost. Infelizmente, como esse encoder atua diretamente nas labels (conjunto y), não é possível torná-lo parte do Pipeline.

In [13]:
xgb_classifier = Pipeline([
    ('discretizer', KBinsDiscretizer(strategy="kmeans")),
    ('clf', XGBClassifier(objective="multi:softmax", num_class=6))
])

xgb_parameters = {
    'discretizer__n_bins': list(range(3, 10)),
    'discretizer__encode': ["ordinal", "onehot"],
    'clf__eta': [0.01, 0.1, 0.2, 0.3],
    'clf__min_child_weight': list(range(1, 10)),
    'clf__max_depth': [None] + list(range(3, 10))
}

label_encoder = LabelEncoder()
label_encoder.fit(y_train.values.ravel())

xgb_randomized_search = RandomizedSearchCV(
    xgb_classifier,
    xgb_parameters,
    scoring=scoring,
    random_state=random_seed,
    n_iter=iterations,
    n_jobs=n_jobs,
    cv=folds
)
xgb_randomized_search.fit(X_train, label_encoder.transform(y_train.values.ravel()))

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('discretizer',
                                              KBinsDiscretizer(strategy='kmeans')),
                                             ('clf',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric=None,
                                                            feature_types=None,
                                                            gamma=None,
                                                            gpu_id=None,
                                                            grow_...
                                                            n_estimators=100,
                                                            n_jobs=None,
                                                            num_class=6,
                                                            num_parallel_tree=None,
                                                            objective='multi:softmax', ...))]),
                   n_jobs=-1,
                   param_distributions={'clf__eta': [0.01, 0.1, 0.2, 0.3],
                                        'clf__max_depth': [None, 3, 4, 5, 6, 7,
                                                           8, 9],
                                        'clf__min_child_weight': [1, 2, 3, 4, 5,
                                                                  6, 7, 8, 9],
                                        'discretizer__encode': ['ordinal',
                                                                'onehot'],
                                        'discretizer__n_bins': [3, 4, 5, 6, 7,
                                                                8, 9]},
                   random_state=42, scoring='accuracy')

In [14]:
xgb_best_params = xgb_randomized_search.best_params_

print("Best parameters for XGBoost")
for param_name in xgb_parameters.keys():
    print(f"{param_name}: {xgb_best_params[param_name]}")

print("Stats for optimized XGBoost")

print("training dataset:")
xgb_train_pred = xgb_randomized_search.predict(X_train)
precision, recall, f1, _ = precision_recall_fscore_support(label_encoder.transform(y_train.values.ravel()), xgb_train_pred)
print("Precision:", precision)
print("recall:", recall)
print("f1:", f1)
print("")

print("testing dataset")
xgb_test_pred = xgb_randomized_search.predict(X_test)
precision, recall, f1, _ = precision_recall_fscore_support(label_encoder.transform(y_test.values.ravel()), xgb_test_pred)
print("Precision:", precision)
print("recall:", recall)
print("f1:", f1)
print("")

Best parameters for XGBoost
discretizer__n_bins: 9
discretizer__encode: ordinal
clf__eta: 0.3
clf__min_child_weight: 9
clf__max_depth: 7
Stats for optimized XGBoost
training dataset:


/home/alps2/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: [0.34530036 0.66849292 0.79203175 0.42346644 0.         0.54338945]
recall: [0.01569655 0.66037939 0.90516894 0.58072522 0.         0.79376744]
f1: [0.03002808 0.66441139 0.84482942 0.48978228 0.         0.64513724]

testing dataset
Precision: [0.3369235  0.66597718 0.79202279 0.41891788 0.         0.5436608 ]
recall: [0.01534032 0.65905946 0.90507801 0.57439802 0.         0.79351992]
f1: [0.02934456 0.66250026 0.84478472 0.48448958 0.         0.64524663]



/home/alps2/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Resultados

## Análise dos resultados do RandomForest
Best parameters for random forest

discretizer__n_bins: 9

discretizer__encode: ordinal

clf__n_estimators: 130

clf__criterion: entropy

clf__max_depth: None

clf__bootstrap: False

Stats for optimized random forest

training dataset:

Precision: [0.34530036 0.6692229  0.79203175 0.42346644 0.         0.54318468]

recall: [0.01569655 0.65931609 0.90516894 0.58072522 0.         0.79459984]

f1: [0.03002808 0.66423256 0.84482942 0.48978228 0.         0.64526753]

testing dataset

Precision: [0.3369235  0.66668469 0.79202279 0.41891788 0.         0.54342545]

recall: [0.01534032 0.65802645 0.90507801 0.57439802 0.         0.79427756]

f1: [0.02934456 0.66232727 0.84478472 0.48448958 0.         0.64533105]

## Análise dos resultados do XGBoost
Best parameters for XGBoost

discretizer__n_bins: 9

discretizer__encode: ordinal

clf__eta: 0.3

clf__min_child_weight: 9

clf__max_depth: 7


Stats for optimized XGBoost

training dataset:

Precision: [0.34530036 0.66849292 0.79203175 0.42346644 0.         0.54338945]

recall: [0.01569655 0.66037939 0.90516894 0.58072522 0.         0.79376744]

f1: [0.03002808 0.66441139 0.84482942 0.48978228 0.         0.64513724]


testing dataset

Precision: [0.3369235  0.66597718 0.79202279 0.41891788 0.         0.5436608 ]

recall: [0.01534032 0.65905946 0.90507801 0.57439802 0.         0.79351992]

f1: [0.02934456 0.66250026 0.84478472 0.48448958 0.         0.64524663]


# Conclusões
